In [1]:
import util as ut
import recomender.trivial as trivial
import evaluate as eval

import json
import numpy as np
from sklearn.model_selection import train_test_split as sk_train_test_split
from operator import itemgetter


In [2]:


# def create_matrix(user_dict, item_dict,  ratings_dict):
#     # # # Create dict for idx of user and item
#     # # user_dict = {}
#     # # item_dict = {}
#     # # user_idx, item_idx = 0, 0
#     # # for user_id, item_id, rating in data:
#     # #     if user_id not in user_dict:
#     # #         user_dict[user_id] = user_idx
#     # #         user_idx += 1
#     # #     if item_id not in item_dict:
#     # #         item_dict[item_id] = item_idx
#     # #         item_idx += 1

#     # Create rating matrix
#     rating_matrix = [[np.nan]*(len(item_dict))]*(len(user_dict)) 
#     print(len(rating_matrix), len(user_dict), len(item_dict))
#     for pair in ratings_dict.keys():
#         user_id, item_id = pair.split(':')
#         # print([user_dict[user_id],item_dict[item_id]])
#         rating_matrix[user_dict[user_id]['Idx']][item_dict[item_id]['Idx']] = ratings_dict[pair]['Rating']

#     return rating_matrix


# train_user_dict, train_item_dict, train_ratings_dict = train
# rating_matrix = create_matrix(train_user_dict, train_item_dict, train_ratings_dict)

In [3]:
# Separate test and train
test, train = ut.read_ratings()

# put test in the same recomendation format
test_recomendation = eval.test_to_recomendation(test)



In [4]:
# def trivial_personalized_prediction(train, targets):
#   # rating_keys = train[2].keys()
#   train_user_dict, train_item_dict, train_ratings_dict = train
  

#   mean_rating = 0
#   for key in train[2]:
#     mean_rating += train[2][key]['Rating']
#   mean_rating = mean_rating/len(train[2])

#   predictions = []
#   for pair in targets:
#     _tuser, _titem = pair.split(':')
#     _dif_item_mean = 0
#     _dif_user_mean = 0
#     if _titem in train_item_dict:
#       _item_dict = train_item_dict[_titem]
#       _dif_item_mean = _item_dict['Rating_sum']/len(_item_dict['Users']) - mean_rating

#     if _tuser in train_user_dict:
#       _user_dict = train_user_dict[_tuser]
#       _dif_user_mean = _user_dict['Rating_sum']/len(_user_dict['Items']) - mean_rating

#     prediction = mean_rating + (_dif_user_mean + _dif_item_mean)/2
#     predictions.append([_tuser, _titem, prediction])
  
#   predictions = sorted(predictions, key=itemgetter(2), reverse=True)  
#   return predictions

# user = predictions = trivial_personalized_prediction(train, test[2].keys())

In [5]:
# def trivial_recomendation(train, targets, user_mean=False):
#   if user_mean: predictions = trivial_personalized_prediction(train, targets)
#   else: predictions = trivial_prediction(train, targets)
#   recomend_dict = {}
  
#   for user, item, rating in predictions:

#     if user not in recomend_dict: recomend_dict[user] = []
#     recomend_dict[user].append(item)

#   with open(ut.RECOMENDATION, 'w') as recomendation_file:
#     recomendations = []
#     for user in recomend_dict.keys():
#       [recomendations.append(user + ':' + item + '\n') for item in recomend_dict[user]]
  
#     recomendation_file.writelines('UserId:ItemId\n')
#     recomendation_file.writelines(recomendations)
#   return recomendations

trivial.trivial_recomendation(train, test[2].keys(), user_mean=True)

['2ea6b5478a:486b54829b\n',
 '2ea6b5478a:823e0c452b\n',
 '2ea6b5478a:8b6785454c\n',
 '2ea6b5478a:bc2ef62f98\n',
 '2ea6b5478a:9253194f71\n',
 '2ea6b5478a:7a40d4e385\n',
 '2ea6b5478a:b6aa949ff3\n',
 '2ea6b5478a:562a55ca69\n',
 '2ea6b5478a:e7df0bd892\n',
 '2ea6b5478a:f2cee9a275\n',
 '2ea6b5478a:f5f6eb0295\n',
 '2ea6b5478a:8edac6fbba\n',
 '2ea6b5478a:9fa80c4aff\n',
 '2ea6b5478a:e1add459d7\n',
 '2ea6b5478a:e7bedb695f\n',
 '2ea6b5478a:f3044ebdbd\n',
 '2ea6b5478a:2187953466\n',
 '2ea6b5478a:78cb280845\n',
 '2ea6b5478a:700e8ac5b7\n',
 '2ea6b5478a:a4142c6864\n',
 '2ea6b5478a:33160a40fd\n',
 '2ea6b5478a:f602311e4e\n',
 '2ea6b5478a:bfc866497a\n',
 '2ea6b5478a:199f8f5ff4\n',
 '2ea6b5478a:50100e4bc4\n',
 '2ea6b5478a:0835347eac\n',
 '2ea6b5478a:1ba96a5b25\n',
 '2ea6b5478a:9d04ee3f9d\n',
 '2ea6b5478a:69f7b4b0e3\n',
 '2ea6b5478a:6556d6184b\n',
 '2ea6b5478a:828c0f5474\n',
 '2ea6b5478a:d36da77803\n',
 '2ea6b5478a:9fba61b7ff\n',
 '2ea6b5478a:8c10ffb5f4\n',
 '2ea6b5478a:467cc05c21\n',
 '2ea6b5478a:a792b29

In [6]:
with open(ut.CONTENT, 'r') as json_file:
    json_list = list(json_file)


content_dict = {}
for json_str in json_list:
    result = json.loads(json_str)
    if (isinstance(result, dict)):
        content_dict[result['ItemId']] = result   